<a href="https://colab.research.google.com/github/CelineBrambles/databricks_status_page_webscraper/blob/main/bronze_databricks_incident_history.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to url> https://status.databricks.com/pages/history/5cf02dde58a00904bda41926

In [2]:

import os
import pandas as pd
import datetime as dt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    # Tell Selenium where Chromium and Chromedriver are (installed by the workflow)
    chrome_options.binary_location = os.getenv("CHROME_BIN", "/usr/bin/chromium-browser")
    service = Service(os.getenv("CHROMEDRIVER_PATH", "/usr/bin/chromedriver"))
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(10)
    return driver


Original

In [3]:
%pip install beautifulsoup4


# WEBSCRAPPING

In [4]:

# Install Google Chrome Stable and Selenium dependencies
!apt-get -y update
!apt-get -y install wget gnupg unzip

# Download & install Google Chrome Stable
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get -y install ./google-chrome-stable_current_amd64.deb || \
  (apt-get -f install -y && apt-get -y install ./google-chrome-stable_current_amd64.deb)

# Install Selenium + webdriver-manager (driver auto-management)
!pip -q install --upgrade selenium webdriver-manager


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,212 B]
Fetched 6,954 B in 1s (4,968 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry mi

In [5]:

import shutil, subprocess

# Verify Chrome install
chrome_path = shutil.which("google-chrome")
print("Chrome binary:", chrome_path)
print("Chrome version:", subprocess.check_output(["google-chrome", "--version"]).decode().strip())

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configure headless Chrome
chrome_options = Options()
chrome_options.binary_location = chrome_path  # make sure Selenium points to the chrome binary
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/120.0 Safari/537.36")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

print("Selenium WebDriver up!")



Chrome binary: /usr/bin/google-chrome
Chrome version: Google Chrome 143.0.7499.169
Selenium WebDriver up!


# GETTING THE MONTH AND YEAR

In [6]:

# Navigate and wait for JS-rendered content
url = "https://status.databricks.com/pages/history/5cf02dde58a00904bda41926"
driver.get(url)

wait = WebDriverWait(driver, 25)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.history_month")))

# Collect month headers
month_divs = driver.find_elements(By.CSS_SELECTOR, "div.history_month")
months = [m.text.strip() for m in month_divs if m.text.strip()]

print("Months found (first 10):", months[:10])
print("Latest month:", months[0] if months else None)

# driver.quit()


Months found (first 10): ['December 2025']
Latest month: December 2025


# GETTING THE INCIDENT NAMES

In [7]:

# from urllib.parse import urljoin
# from selenium.webdriver.common.by import By

# # Reuse the same driver & wait; page already loaded and months found
# # Ensure at least one incident link is present before collecting
# wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/pages/incident/']")))

# BASE = "https://status.databricks.com"
# anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/pages/incident/']")

# incidents = []
# for a in anchors:
#     name = (a.text or "").strip()
#     href = a.get_attribute("href") or ""
#     if not href:
#         continue
#     # Make absolute if the href is relative (starts with "/")
#     url_abs = urljoin(BASE, href)
#     if name:
#         incidents.append({"name": name, "url": url_abs})

# print(f"Found {len(incidents)} incident links")
# for row in incidents[:10]:
#     print(row)


In [8]:

# from selenium.webdriver.common.by import By
# import time
# from urllib.parse import urljoin

# BASE = "https://status.databricks.com"

# def count_incidents_js(drv):
#     # Count anchors that point to incident pages
#     return drv.execute_script("return document.querySelectorAll(\"a[href*='/pages/incident/']\").length;")

# # 1) Count current incidents
# before_count = count_incidents_js(driver)
# print("Incident count before click:", before_count)

# # 2) Click the Show More button by ID (your markup)
# try:
#     btn = driver.find_element(By.ID, "showMoreBtn1")
#     # Scroll into view and click via JS for robustness
#     driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
#     driver.execute_script("arguments[0].click();", btn)
# except Exception as e:
#     print("Could not click Show More by ID:", e)
#     # Fallback: try class-based selector if ID changes
#     try:
#         btn2 = driver.find_element(By.CSS_SELECTOR, "button.showMoreBtn.showMore")
#         driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn2)
#         driver.execute_script("arguments[0].click();", btn2)
#     except Exception as e2:
#         print("Fallback click failed:", e2)

# # 3) Wait briefly for new items to render
# time.sleep(1.0)

# # Optional: loop a few times until the count increases
# max_wait_secs = 6
# deadline = time.time() + max_wait_secs
# after_count = count_incidents_js(driver)
# while after_count <= before_count and time.time() < deadline:
#     time.sleep(0.5)
#     after_count = count_incidents_js(driver)

# print("Incident count after click:", after_count)

# # 4) Re-collect ALL incident links (including text-red/text-orange)
# anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*='/pages/incident/']")
# incidents_all = []
# for a in anchors:
#     name = (a.text or "").strip()  # Includes nested <span> text
#     href = a.get_attribute("href") or ""
#     if not href or not name:
#         continue
#     incidents_all.append({"name": name, "url": urljoin(BASE, href)})

# print(f"Total incidents collected: {len(incidents_all)}")
# for row in incidents_all:
#     print(row)

# # 5) (Optional) show only newly added ones after Show More
# before_set = set()  # if you saved 'incidents' from the previous scrape:
# # Example: before_set = {x['url'] for x in incidents}  # uncomment and set if you kept earlier results

# new_ones = [x for x in incidents_all if x['url'] not in before_set]
# print("\nNew incidents after Show More:")
# for row in new_ones:
#     print(row)


# GETTING INCIDENTS NAMES AND DESCRIPTIONS

In [9]:

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# Assumes `driver` and `wait` are already set and the page is loaded.
url = "https://status.databricks.com/pages/history/5cf02dde58a00904bda41926"
driver.get(url)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.history_month")))

# 1) Locate the December 2025 section and get its container
dec_div = driver.find_element(
    By.XPATH,
    "//div[contains(@class,'history_month') and normalize-space(.)='December 2025']"
)
container = driver.execute_script("return arguments[0].nextElementSibling;", dec_div)

# 2) Use JS to extract name + (hidden) URL + description from that container
items = driver.execute_script("""
  const base = 'https://status.databricks.com';
  const root = arguments[0];
  if (!root) return [];

  // Each incident usually has: anchor(s), a hidden span with URL, and a div.event_description
  const descriptions = Array.from(root.querySelectorAll('div.event_description'));

  return descriptions.map(desc => {
    // Try to find the main anchor near the description
    // Prefer the sibling/ancestor link with '/pages/incident/'
    let a = null;
    // previous sibling or parent subtree
    const candidateLinks = [
      desc.previousElementSibling,
      desc.parentElement
    ].flatMap(el => el ? Array.from(el.querySelectorAll('a[href*="/pages/incident/"]')) : []);

    if (candidateLinks.length > 0) a = candidateLinks[0];

    // Fallback: search upward for any related link
    if (!a) {
      const upLinks = Array.from(desc.closest('li, div, section').querySelectorAll('a[href*="/pages/incident/"]'));
      if (upLinks.length) a = upLinks[0];
    }

    const name = a ? (a.textContent || '').trim() : '';
    // Hidden URL (relative) often in a <span class="is-hidden">.../pages/incident/...<span>
    let rel = null;
    const hidden = desc.parentElement ? desc.parentElement.querySelector('span.is-hidden') : null;
    if (hidden) rel = (hidden.textContent || '').trim();

    // If hidden span not found, use the anchor href
    let href = rel && rel.startsWith('/') ? (base + rel) : (a ? a.getAttribute('href') : null);
    if (href && href.startsWith('/')) href = base + href;

    const description = (desc.textContent || '').trim();

    return { name, url: href || null, event_description: description };
  });
""", container)

# 3) Print name + description (no saving)
for i, it in enumerate(items, 1):
    print(f"{i}. {it['name']}")
    print(f"   Description: {it['event_description']}\n")


1. US West 2 - Compute (Degraded Performance)
   Description: Between 00:11 UTC and 01:32 UTC December 17, 2025, customers may have experienced failures or delays when provisioning classic compute resources for interactive or job workloads.

The incident is now closed. We apologize for any inconvenience this may have caused. If you require further details about this incident, please submit a support ticket or contact help@databricks.com.

2. US East 1 - Compute
   Description: Our investigation has confirmed that this issue did not impact the Databricks service. 

We are closing this incident and apologize for any confusion.

3. US Gov West 1 - Multiple services (Service Disruption)
   Description: Between 23:51 UTC on December 3 and 01:00 UTC on December 4, 2025, customers may have experienced a complete outage, with all Databricks services presumed unavailable during this period.

The incident is now closed. We apologize for any inconvenience this may have caused. If you require furt

In [10]:

import re
import pandas as pd

# 1) Parse the latest month string from the already-retrieved variable
#    e.g., months = ["December 2025"]
latest = months[0].strip()
m = re.match(r"^([A-Za-z]+)\s+(\d{4})$", latest)
if not m:
    raise ValueError(f"Cannot parse month/year from: {latest}")
month_name, year_val = m.group(1), int(m.group(2))

# 2) Build rows from your 'items' list (name + description already retrieved)
rows = []
for it in items:
    rows.append({
        "year": year_val,
        "month": month_name,
        "incident_name": (it.get("name") or "").strip(),
        "incident_description": (it.get("event_description") or "").strip()
    })

# 3) Create DataFrame and save CSV (no index)
df = pd.DataFrame(rows, columns=["year", "month", "incident_name", "incident_description"])
print(f"Total rows: {len(df)}")
display(df.head(10))

df.to_csv("databricks_incidents.csv", index=False)
print("Saved: databricks_incidents.csv")


Total rows: 3


,year,month,incident_name,incident_description
0,2025,December,US West 2 - Compute (Degraded Performance),"Between 00:11 UTC and 01:32 UTC December 17, 2..."
1,2025,December,US East 1 - Compute,Our investigation has confirmed that this issu...
2,2025,December,US Gov West 1 - Multiple services (Service Dis...,Between 23:51 UTC on December 3 and 01:00 UTC ...


Saved: databricks_incidents.csv


In [11]:

# Colab: install the Databricks SQL connector
!pip install databricks-sql-connector


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.9 MB/s eta 0:00:00
  Created wheel for thrift: filename=thrift-0.20.0-cp312-cp312-linux_x86_64.whl size=415378 sha256=3509b4606cbc85b12c9b903ce34360c31ddde756b1dc8db6be21e866b1cd38f5
  Stored in directory: /root/.cache/pip/wheels/57/4d/9e/539ef8e052f8e2db244ddf8ac1f2e623b4663f2ee233ffafb1
Successfully built thrift
